In [1]:
import time
import copy
import numpy as np
from skimage.filters import gaussian
import cv2 

import matplotlib.pyplot as plt
import pylab as pl

import itertools

%matplotlib qt5
#%pylab inline
# pylab.rcParams['figure.figsize'] = (15, 10)

from tifffile import imread, imsave
import pickle

from skimage.draw import circle

import csv

from skimage import io
from scipy.interpolate import griddata
from sklearn.neighbors import KDTree
import math

In [2]:
filename = '/Users/prakash/Desktop/imageForTPS.tif'
flow_channel = imread(filename)
flow_channel.shape

(2, 836, 628)

In [3]:
with open('/Users/prakash/Desktop/MarkGT/Tracks_with_uncertainty_radius.csv', 'r') as f:
    reader = csv.reader(f)
    uncertainty_radius_list_full = list(reader)
uncertainty_radius_list_full = uncertainty_radius_list_full[1:]
uncertainty_radius_list_full=np.array(uncertainty_radius_list_full,np.int)
uncertainty_radius_list_full = [item for sublist in uncertainty_radius_list_full for item in sublist]
# print(uncertainty_radius_list_full)

In [4]:
uncertainty_radius_list_full = np.array(uncertainty_radius_list_full)
zero_uncertainty_index = np.where(uncertainty_radius_list_full == 0)[0]
nonzero_uncertainty_index = np.where(uncertainty_radius_list_full !=0)[0]

In [5]:
zero_uncertainty_index

array([ 19,  20,  21,  22,  24,  25,  28,  29,  31,  32,  33,  47,  48,
        58,  76,  77,  89, 102, 103, 104, 105, 112, 124, 125, 126, 127,
       128, 129, 130, 131, 139, 140, 141])

In [6]:
### Reading only certain indices (with uncertainty readius = 0) for warping
with open('/Users/prakash/Desktop/MarkGT/Choices_Structured.csv') as sd:
    source_reader=csv.reader(sd)
    next(source_reader)
    certain_source_rows=[row for idx, row in enumerate(source_reader) if idx in zero_uncertainty_index]
    certain_source_rows = np.array(certain_source_rows,int)

with open('/Users/prakash/Desktop/MarkGT/Tracks_Structured.csv') as td:
    target_reader=csv.reader(td)
    next(target_reader)
    certain_target_rows=[row for idx, row in enumerate(target_reader) if idx in zero_uncertainty_index]
    certain_target_rows = np.array(certain_target_rows,int)

In [7]:
# def compute_flow( flowchannel ):
#     '''Computes the Farnaback dense flow for the given moview
#     '''
#     flows = []
#     prvs = flowchannel[0]
    
   
#     for f in range(flowchannel.shape[0]-1):
#         print(f)
#         nxt = flowchannel[f]
    
#         flow = cv2.calcOpticalFlowFarneback(prev=prvs,
#                                             next=nxt,
#                                             flow=None,
#                                             pyr_scale=0.5, 
#                                             levels=1,
#                                             winsize=5, #15?
#                                             iterations=2,
#                                             poly_n=5, 
#                                             poly_sigma=1.1, 
#                                             flags=0)
      
#         flows.append(flow)
#         prvs = nxt
#         print ('.', end="")
#     print (' ...done!')

#     flow_x, flow_y = split_flow_components( flows )
#     flows=[flow_x, flow_y]
#     return flows

In [8]:
# def compute_flow_TPS( flowchannel,pos_s,pos_t):
#     '''Computes the Thin Plate Splines for the given movie
#     '''
#     src_img = flowchannel[0]
#     des_img = flowchannel[1]
    
#     tps = cv2.createThinPlateSplineShapeTransformer()
    
#     sshape = pos_s.astype(np.float32)
#     tshape = pos_t.astype(np.float32)
#     sshape = sshape.reshape(1,-1,2)
#     tshape = tshape.reshape(1,-1,2)
    
    
#     matches = list()
#     for i in range(0, sshape.shape[1],1):
#         matches.append(cv2.DMatch(i,i,0))
        
        
        
#     tps.estimateTransformation(sshape,tshape,matches)

#     Xpts = np.arange(0, src_img.shape[0])
#     Ypts = np.arange(0, src_img.shape[1])
#     Points = np.array(list(itertools.product(Xpts, Ypts))).astype( np.float32)
    
#     ret, tshape_ = tps.applyTransformation (Points.reshape(1,-1,2))
#     Point_skrewd = tshape_.reshape(src_img.shape[0],src_img.shape[1],2)    
    
    
#     Points=Points.reshape(src_img.shape[0],src_img.shape[1],2)    
    
    
#     flows=Point_skrewd-Points
    
#     return [flows]

In [9]:
# def compute_interpolated_flow( flowchannel,pos_s,pos_t): /// Maybe wrong !!!
#     '''Computes the Thin Plate Splines for the given movie
#     '''
#     src_img = flowchannel[0]
#     x = pos_s[:,0]
#     y = pos_s[:,1]
#     fx = pos_t[:,0]-pos_s[:,0]
#     fy = pos_t[:,1]-pos_s[:,1]
#     img_shape=np.shape(src_img)
    
    
#     xy=np.stack((x,y),axis=1)
    
#     mesh_x = np.arange(np.shape(flowchannel[0])[0])
#     mesh_y = np.arange(np.shape(flowchannel[0])[1])
#     mesh_y, mesh_x = np.meshgrid(mesh_x, mesh_y)
    
#     mesh_xy=np.stack((mesh_x.flatten(),mesh_y.flatten()),axis=1)
    
#     u=griddata(xy, fx, mesh_xy, method='nearest')
#     v=griddata(xy, fy, mesh_xy, method='nearest')
# #     u=griddata(xy, fx, mesh_xy, method='linear')
# #     v=griddata(xy, fy, mesh_xy, method='linear')
#     u = u.reshape(src_img.shape[0],src_img.shape[1])
#     v = v.reshape(src_img.shape[0],src_img.shape[1])
    
#     print(u.shape)
    
#     flows=np.stack((u,v),axis=2)
    
#     return [flows]

In [10]:
# def kNN_weight_definition(k_value, distance, neighbor_index):
#     """ Finds kNN depending on user specified k and defines weights for these neighbors inversly proportional to the 
#         distance of neighbors from the queried point.
#     """
#     weight_list = []
#     normalized_weight_vector = []
#     sum_of_weights = 0
#     distance_squared = distance*distance
#     distance_inverse = 1/(distance_squared+0.001)
    
#     for i in range (0,k_value):
#         weight =  distance_inverse[0][i]
#         sum_of_weights =  sum_of_weights+weight
#         weight_list.append(weight)
        
#     for j in range(0,len(weight_list)):
#         normalized_weight = weight_list[j]/sum_of_weights
#         normalized_weight_vector.append(normalized_weight)
        
#     return normalized_weight_vector


# def compute_kNNinterpolated_flow( flowchannel,pos_s,pos_t):
#     '''k Nearest neighbor based interpolation
#     '''
#     k =5
#     x = pos_s[:,0]
#     y = pos_s[:,1]
#     fx = pos_t[:,0]-pos_s[:,0]
#     fy = pos_t[:,1]-pos_s[:,1]
    
#     mesh_x = np.arange(np.shape(flowchannel[0])[0])
#     mesh_y = np.arange(np.shape(flowchannel[0])[1])
#     mesh_y, mesh_x = np.meshgrid(mesh_x, mesh_y)
    
#     u=[]
#     v=[]
#     mesh_xy=np.stack((mesh_x.flatten(),mesh_y.flatten()),axis=1)
#     tree = KDTree(np.stack((x,y),axis=1))
#     for index in range(len(mesh_xy)):
#         pt = mesh_xy[index,:]   # <-- the point to find
#         distance, neighbor_index = tree.query([pt], k) 
#         normalized_weight_vector = kNN_weight_definition (k, distance, neighbor_index)
        
#         u.append(np.sum(fx[neighbor_index]*normalized_weight_vector))
#         v.append(np.sum(fy[neighbor_index]*normalized_weight_vector))
        
#     u = np.array(u)
#     v = np.array(v)
#     u = u.reshape(flowchannel[0].shape[0],flowchannel[0].shape[1])
#     v = v.reshape(flowchannel[0].shape[0],flowchannel[0].shape[1])    
#     flows=np.stack((u,v),axis=2)
    
#     return [flows]

In [11]:
def kNN_weight_definition(k_value, distance, neighbor_index):
    """ Finds kNN depending on user specified k and defines weights for these neighbors inversly proportional to the 
        distance of neighbors from the queried point.
    """
    weight_list = []
    normalized_weight_vector = []
    sum_of_weights = 0
    distance_squared = distance*distance
    distance_inverse = 1/(distance_squared+0.001)
    
    for i in range (0,k_value):
        weight =  distance_inverse[0][i]
        sum_of_weights =  sum_of_weights+weight
        weight_list.append(weight)
        
    for j in range(0,len(weight_list)):
        normalized_weight = weight_list[j]/sum_of_weights
        normalized_weight_vector.append(normalized_weight)
        
    return normalized_weight_vector


def compute_kNNinterpolated_flow_With_OF( flowchannel,pos_s,pos_t):
    '''k Nearest neighbor based interpolation
    
    '''
    flow = cv2.calcOpticalFlowFarneback(prev=flowchannel[0],
                                            next=flowchannel[1],
                                            flow=None,
                                            pyr_scale=0.5, 
                                            levels=1,
                                            winsize=5, #15?
                                            iterations=2,
                                            poly_n=5, 
                                            poly_sigma=1.1, 
                                            flags=0)
    flow_x, flow_y = split_flow_components( flow )
    flow_xy=np.stack((flow_x.flatten(),flow_y.flatten()),axis=1)
    print(flow_x.shape)
    print(flow_y.shape)
    
    
    k =5
    x = pos_s[:,0]
    y = pos_s[:,1]
    fx = pos_t[:,0]-pos_s[:,0]
    fy = pos_t[:,1]-pos_s[:,1]
    
    flow_x_pos=flow_x[x,y]
    flow_y_pos=flow_y[x,y]
    
    
    mesh_x = np.arange(np.shape(flowchannel[0])[0])
    mesh_y = np.arange(np.shape(flowchannel[0])[1])
    mesh_y, mesh_x = np.meshgrid(mesh_x, mesh_y)
    
    u=[]
    v=[]
    mesh_xy=np.stack((mesh_x.flatten(),mesh_y.flatten()),axis=1)
    tree = KDTree(np.stack((x,y),axis=1))
    for index in range(len(mesh_xy)):
        pt = mesh_xy[index,:]   # <-- the point to find
        distance, neighbor_index = tree.query([pt], k) 
        normalized_weight_vector = kNN_weight_definition (k, distance, neighbor_index)
        
        
        
        u_tmp=np.sum(fx[neighbor_index]*normalized_weight_vector)
        v_tmp=np.sum(fy[neighbor_index]*normalized_weight_vector)
        
        u.append(u_tmp)
        v.append(v_tmp)
        
    u = np.array(u)
    v = np.array(v)
    u = u.reshape(flowchannel[0].shape[0],flowchannel[0].shape[1])
    v = v.reshape(flowchannel[0].shape[0],flowchannel[0].shape[1])    
    flows=np.stack((u,v),axis=2)
    
#     return [flows]

In [12]:
def split_flow_components( flows ):
    ''' Receives flow results as computed by 'compute_flow' and returns 
        a tupel of x and y components of this flow results.
    '''
    flow_x = np.moveaxis(np.swapaxes(flows,0,3)[0],-1,0)
    flow_y = np.moveaxis(np.swapaxes(flows,0,3)[1],-1,0)
    return flow_x, flow_y



In [13]:
#### Quantification
def quantify_flows(flows,validation_source_rows,validation_target_rows):
    
    flow_x=flows[0]
    flow_x=flows[1]
    u = flow_x[0,validation_source_rows[:,0],validation_source_rows[:,1]]
    v = flow_y[0,validation_source_rows[:,0],validation_source_rows[:,1]]
    tmp=validation_target_rows-validation_source_rows
    u0=tmp[:,0]
    v0=tmp[:,1]
    
    len_vec=(u**2+v**2)**0.5
    len_vec0=(u0**2+v0**2)**0.5
    
    euk=((u-u0)**2+(v-v0)**2)**0.5
    
    angle=np.arccos((u*u0+v*v0)/(len_vec*len_vec0+np.finfo(float).eps))/np.pi*180
    
    lag=len_vec-len_vec0
    
    euk_rel= (euk/len_vec0)*100
    lag_rel = (lag/len_vec0)*100
    
    return lag, euk, angle, euk_rel, lag_rel
    

In [14]:
k = 11
np.random.seed(0)
r=np.random.choice(len(certain_source_rows), size=len(certain_source_rows), replace=False, p=None)

In [15]:
full_angle = []
full_lag = []
full_euk = []
full_euk_rel = []
full_lag_rel = []

certain_source_rows = certain_source_rows[r,:]
certain_target_rows = certain_target_rows[r,:]
step = int(np.ceil(len(certain_source_rows)/k))
for i in range(k):
    k_fold_validation_indices = np.arange(i*step,i*step+step)
    k_fold_training_indices = np.concatenate((np.arange(i*step),np.arange(i*step+step,len(certain_source_rows))),axis=0)
    training_source_rows = certain_source_rows[k_fold_training_indices,:]
    training_target_rows = certain_target_rows[k_fold_training_indices,:]
    validation_source_rows = certain_source_rows[k_fold_validation_indices,:]
    validation_target_rows = certain_target_rows[k_fold_validation_indices,:]
    
    ####Change this optical flow, it's just for testing pipeline
    flows = compute_kNNinterpolated_flow_With_OF(flow_channel, training_source_rows, training_target_rows) # It should be a list of concatenated u,v.
    lag, euk, angle, euk_rel, lag_rel = quantify_flows(flows,validation_source_rows,validation_target_rows)
    full_angle.append(list(angle))
    full_lag.append(list(lag))
    full_euk.append(list(euk))
    full_lag_rel.append(list(lag_rel))
    full_euk_rel.append(list(euk_rel))
    
#     tmp=validation_target_rows-validation_source_rows
#     u0=tmp[:,0]
#     v0=tmp[:,1]
    
    
    

ValueError: bad axis2 argument to swapaxes

In [ ]:
print("Angle difference:", np.median(full_angle))
print("Pixel lag:", np.median(np.abs(full_lag)))
print("Euclidean offset:", np.median(full_euk))
print("Relative euclidean offset:", np.median(full_euk_rel))
print("Relative lag:", np.median(np.abs(full_lag_rel)))

In [ ]:
x = np.arange(np.shape(flow_channel[0])[0])
y = np.arange(np.shape(flow_channel[0])[1])
y, x = np.meshgrid(x, y)
u=flows[0][:,:,0]
v=flows[0][:,:,1]

x=x[0::10,0::10]
y=y[0::10,0::10]
u=u[0::10,0::10]
v=v[0::10,0::10]

plt.imshow(flow_channel[0])

plt.quiver(x,y,u,v)

plt.show()